<a href="https://colab.research.google.com/github/armandoordonez/eda_couse/blob/main/One_hot_encodign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Based on: https://www.analyticsvidhya.com/blog/2020/03/one-hot-encoding-vs-label-encoding-using-scikit-learn/
# https://www.geeksforgeeks.org/detecting-multicollinearity-with-vif-python/



# importamos librería y leemos el dataset

import pandas as pd
import numpy as np

df=pd.read_csv("https://raw.githubusercontent.com/armandoordonez/eda_couse/main/data/salary.csv")
df.head()
    

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes


In [17]:
df.shape

(10, 4)

In [18]:
print(df.info)


<bound method DataFrame.info of    Country   Age   Salary Purchased
0   France  44.0  72000.0        No
1    Spain  27.0  48000.0       Yes
2  Germany  30.0  54000.0        No
3    Spain  38.0  61000.0        No
4  Germany  40.0      NaN       Yes
5   France  35.0  58000.0       Yes
6    Spain   NaN  52000.0        No
7   France  48.0  79000.0       Yes
8  Germany  50.0  83000.0        No
9   France  37.0  67000.0       Yes>


In [19]:
# importing one hot encoder 
from sklearn.preprocessing import OneHotEncoder

# Creamos el objeto
onehotencoder = OneHotEncoder()

#reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object 

X = onehotencoder.fit_transform(df.Country.values.reshape(-1,1)).toarray()

X


array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [20]:
# Agregamos al dataframe original 

dfOneHot = pd.DataFrame(X, columns = ["Country_"+str(int(i)) for i in range(X.shape[1])]) 

In [21]:

df = pd.concat([df, dfOneHot], axis=1)
df


,Country,Age,Salary,Purchased,Country_0,Country_1,Country_2
0,France,44.0,72000.0,No,1.0,0.0,0.0
1,Spain,27.0,48000.0,Yes,0.0,0.0,1.0
2,Germany,30.0,54000.0,No,0.0,1.0,0.0
3,Spain,38.0,61000.0,No,0.0,0.0,1.0
4,Germany,40.0,NaN,Yes,0.0,1.0,0.0
5,France,35.0,58000.0,Yes,1.0,0.0,0.0
6,Spain,NaN,52000.0,No,0.0,0.0,1.0
7,France,48.0,79000.0,Yes,1.0,0.0,0.0
8,Germany,50.0,83000.0,No,0.0,1.0,0.0
9,France,37.0,67000.0,Yes,1.0,0.0,0.0


In [22]:
#droping the country column 
df= df.drop(['Country'], axis=1) 

#printing to verify 
df.head()

,Age,Salary,Purchased,Country_0,Country_1,Country_2
0,44.0,72000.0,No,1.0,0.0,0.0
1,27.0,48000.0,Yes,0.0,0.0,1.0
2,30.0,54000.0,No,0.0,1.0,0.0
3,38.0,61000.0,No,0.0,0.0,1.0
4,40.0,NaN,Yes,0.0,1.0,0.0


In [23]:
# Eliminamos los nulos
df = df.dropna()

In [24]:
df.head()

,Age,Salary,Purchased,Country_0,Country_1,Country_2
0,44.0,72000.0,No,1.0,0.0,0.0
1,27.0,48000.0,Yes,0.0,0.0,1.0
2,30.0,54000.0,No,0.0,1.0,0.0
3,38.0,61000.0,No,0.0,0.0,1.0
5,35.0,58000.0,Yes,1.0,0.0,0.0


In [31]:
# Calculamos Variance Inflation Factor(VIF).
# Cuantifica la  multicolinealidad en un análisis de regresión normal de mínimos cuadrados.
# VIF=1, Baja Multicollinearidad
# VIF=1 VIF<5, Moderada Multicollinearidad
# VIF=1 VIF>5, Multicollinearity Extrema (Se busca evitar)



import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [28]:
# Seleccionamos las variables que vamos a analizar 
X = df[['Salary','Country_0', 'Country_1', 'Country_2']]
  
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature

vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
print(vif_data)

     feature        VIF
0     Salary   1.417456
1  Country_0  26.769959
2  Country_1  13.698917
3  Country_2   9.038566


* VIF=1, Baja Multicollinearidad
* VIF<5, Moderada Multicollinearidad
* VIF>5, Multicollinearity Extrema (Se busca evitar)

In [29]:
# Eliminamos una columna y verificamos el impacto
X= X.drop(['Country_2'], axis=1) 

In [30]:
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature

vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
print(vif_data)

     feature       VIF
0     Salary  5.256082
1  Country_0  3.851112
2  Country_1  2.404970
